## Data and library imports

In [141]:
import pandas as pd
import re

In [2]:
# mapping files
old_g2e = pd.read_csv("/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/qtl/eqtl/annotations/g2e2d_1000IBD_dedup.tsv", sep='\t', names=(['exp', 'geno','study']))
sample_names = pd.read_csv("/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/paper_genotype/1000IBD_sample_names.txt", names=(['geno']))
gem_dna_sample_mapping = pd.read_csv("/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/paper_genotype/mapping/gem_dna_sample_mapping.tsv", sep='\t')
gem_rna_sample_mapping = pd.read_csv("/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/paper_genotype/mapping/gem_rna_sample_mapping.tsv", sep='\t')

In [61]:
# 1000IBD expression file
exp = pd.read_csv("/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/rna/qc/output/9_covariate_correction/1000IBD_gene_counts-TMM.SampleSelection.ProbesWithZeroVarianceRemoved.Log2Transformed.forcenormal.covariatecorrected.txt.gz.CovariatesRemovedOLS.txt.gz", sep="\t")

## Data inspection

In [3]:
old_g2e

,exp,geno,study
0,214-1378,R103_B1289_1A,1000IBD
1,214-1453,R467_B_815_1B,1000IBD
2,214-1686,R161_B586_1,1000IBD
3,214-1405,R112_B1165_1A,1000IBD
4,214-1800,R575_B_695_3A,1000IBD
...,...,...,...
222,215-1719,R489_B_1345_2A,1000IBD
223,215-1610,R40_B222_1,1000IBD
224,215-1739,R138_B828_2A,1000IBD
225,216-3164,R422_B_431_2A,1000IBD


In [4]:
sample_names

,geno
0,mis-ibd-PSIc-mix-mv-Il5M*208-2491_208-2491
1,mis-ibd-PSIc-mix-mv-Il5M*208-2493_208-2493
2,mis-ibd-PSIc-mix-mv-Il5M*208-2494_208-2494
3,mis-ibd-PSIc-mix-mv-Il5M*208-2512_208-2512
4,mis-ibd-PSIc-mix-mv-Il5M*208-2514_208-2514
...,...
3673,mis-ibd-PSIc-mix-mv-Il5M*S5677I.Z0_S5677I.Z0
3674,mis-ibd-PSIc-mix-mv-Il5M*S5678I.Z0_S5678I.Z0
3675,mis-ibd-PSIc-mix-mv-Il5M*S5679I.Z0_S5679I.Z0
3676,mis-ibd-PSIc-mix-mv-Il5M*S5680I.Z0_S5680I.Z0


In [5]:
gem_dna_sample_mapping

,DNA sample ID,Research ID
0,210-22055,UMCGIBD00747
1,208-5216,UMCGIBD00825
2,210-21854,UMCGIBD00935
3,210-22200,UMCGIBD00947
4,208-5240,UMCGIBD00962
...,...,...
1134,214-1453,UMCGIBD00006
1135,214-1378,UMCGIBD00005
1136,214-1687,UMCGIBD00004
1137,216-3027,UMCGIBD00003


In [6]:
gem_rna_sample_mapping

,Research ID,scopy date,biopsy research number,Novogene ID,Biopt location,Cohort,Sequencing batch,Opmerking
0,UMCGIBD00279,19-06-2006,525-3A,545_Biopsy_525-3A,NO coecum,1000IBD selection,Novogene 2020,NaN
1,UMCGIBD00279,19-06-2006,525-1,642_Biopsy_525-1,NO ileum,1000IBD selection,Novogene 2020,NaN
2,UMCGIBD00279,19-06-2006,525-2,91_Biopsy_525-2,O transversum,1000IBD selection,Novogene 2020,NaN
3,UMCGIBD00455,11/01/2006,574-1,45_Biopsy_574-1,NO ileum,1000IBD selection,Novogene 2020,NaN
4,UMCGIBD00455,11/01/2006,574-3,628_Biopsy_574-3,NO transversum,1000IBD selection,Novogene 2020,NaN
...,...,...,...,...,...,...,...,...
981,UMCGIBD01625,07-03-2019,1707-2A,n.v.t.,O descendens,Inf/NotInf Werna,alone 16S,NaN
982,UMCGIBD01759,10-09-2019,1858-A/B,n.v.t.,O sigmoid,VEDO Valerie,alone 16S,NaN
983,UMCGIBD00384,02-06-2014,1235-1B,n.v.t.,NO transversum,1000IBD selection,UMCG 2016,Sample ID komt overeen met sample ID van ander...
984,UMCGIBD00384,02-06-2014,1235-2B,n.v.t.,O sigmoid,1000IBD selection,UMCG 2016,Sample ID komt overeen met sample ID van ander...


## Data manipulation

In [53]:
sample_names_copy = sample_names.copy()

In [54]:
sample_names_copy["geno_short"] = sample_names_copy["geno"].str.replace("mis-ibd-PSIc-mix-mv-Il5M*", "")
sample_names_copy[['geno_id', 'geno_id_duplicate']] = sample_names_copy["geno_short"].str.split("_", expand=True)

In [56]:
print(len(set(sample_names_copy['geno_id']) & set(sample_names_copy['geno_id_duplicate'])))
sample_names["geno"].str.contains("mis-ibd-PSIc-mix-mv-Il5M*").value_counts()

3678


geno
True    3678
Name: count, dtype: int64

In [57]:
sample_names_copy.drop(columns=["geno_short", "geno_id_duplicate"], inplace=True)

In [111]:
exp_samples = exp.columns.tolist()[1::]
df_exp_samples = pd.DataFrame({'exp_id':exp_samples})
df_exp_samples

,exp_id
0,R690_B_1660_1
1,R501_B_1001_1A
2,R522_B_1418_3A
3,R601_B_734_3A
4,R589_B_1438_1A
...,...
659,R357_B1219_3B
660,R160_B278_1
661,R296_B1481_3A
662,R646_B_1754_2


## Merging

In [59]:
merged_df = pd.merge(
    sample_names_copy,
    gem_dna_sample_mapping,
    left_on="geno_id",
    right_on="DNA sample ID",
    how="inner"
)

merged_df

,geno,geno_id,DNA sample ID,Research ID
0,mis-ibd-PSIc-mix-mv-Il5M*208-2491_208-2491,208-2491,208-2491,UMCGIBD00165
1,mis-ibd-PSIc-mix-mv-Il5M*208-2493_208-2493,208-2493,208-2493,UMCGIBD00943
2,mis-ibd-PSIc-mix-mv-Il5M*208-2494_208-2494,208-2494,208-2494,UMCGIBD01260
3,mis-ibd-PSIc-mix-mv-Il5M*208-2512_208-2512,208-2512,208-2512,UMCGIBD01036
4,mis-ibd-PSIc-mix-mv-Il5M*208-2514_208-2514,208-2514,208-2514,UMCGIBD00061
...,...,...,...,...
1066,mis-ibd-PSIc-mix-mv-Il5M*216-3247_216-3247,216-3247,216-3247,UMCGIBD01422
1067,mis-ibd-PSIc-mix-mv-Il5M*216-3249_216-3249,216-3249,216-3249,UMCGIBD00286
1068,mis-ibd-PSIc-mix-mv-Il5M*216-3250_216-3250,216-3250,216-3250,UMCGIBD01458
1069,mis-ibd-PSIc-mix-mv-Il5M*216-3251_216-3251,216-3251,216-3251,UMCGIBD01494


In [60]:
merged_df_final = pd.merge(
    merged_df,
    gem_rna_sample_mapping,
    left_on="Research ID",
    right_on="Research ID"
)

merged_df_final

,geno,geno_id,DNA sample ID,Research ID,scopy date,biopsy research number,Novogene ID,Biopt location,Cohort,Sequencing batch,Opmerking
0,mis-ibd-PSIc-mix-mv-Il5M*208-2512_208-2512,208-2512,208-2512,UMCGIBD01036,03/03/2008,713-1B,367_Biopsy_713-1B,NO transversum,1000IBD selection,Novogene 2020,NaN
1,mis-ibd-PSIc-mix-mv-Il5M*209-0014_209-0014,209-0014,209-0014,UMCGIBD00912,08-11-2010,1004-1A,11_Biopsy_1004-1A,NO ileum,1000IBD selection,Novogene 2020,NaN
2,mis-ibd-PSIc-mix-mv-Il5M*209-0014_209-0014,209-0014,209-0014,UMCGIBD00912,08-11-2010,1004-2A,154_Biopsy_1004-2A,NO ascendens,1000IBD selection,Novogene 2020,NaN
3,mis-ibd-PSIc-mix-mv-Il5M*209-0014_209-0014,209-0014,209-0014,UMCGIBD00912,08-11-2010,1004-3A,81_Biopsy_1004-3A,O sigmoid,1000IBD selection,Novogene 2020,NaN
4,mis-ibd-PSIc-mix-mv-Il5M*209-0035_209-0035,209-0035,209-0035,UMCGIBD00599,08/03/2007,659-4A,171_Biopsy_659-4A,O descendens,1000IBD selection,Novogene 2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...
685,mis-ibd-PSIc-mix-mv-Il5M*216-3244_216-3244,216-3244,216-3244,UMCGIBD01532,09-05-2016,1393-3A,52_Biopsy_1393-3A,O rectum/sigmoid,VEDO Valerie Colij,Novogene 2020,NaN
686,mis-ibd-PSIc-mix-mv-Il5M*216-3244_216-3244,216-3244,216-3244,UMCGIBD01532,09-05-2016,1393-1A,554_Biopsy_1393-1A,O ileum,VEDO Valerie Colij,Novogene 2020,NaN
687,mis-ibd-PSIc-mix-mv-Il5M*216-3247_216-3247,216-3247,216-3247,UMCGIBD01422,29-01-2019,1673-1,351_Biopsy_1673-1,O coecum,Inf/NotInf Werna,Novogene 2020,NaN
688,mis-ibd-PSIc-mix-mv-Il5M*216-3247_216-3247,216-3247,216-3247,UMCGIBD01422,29-01-2019,1673-2,519_Biopsy_1673-2,NO ascendens,Inf/NotInf Werna,Novogene 2020,NaN


In [67]:
# checing how many unique patients we are dealing with
merged_df_final["geno"].value_counts()

geno
mis-ibd-PSIc-mix-mv-Il5M*214-1292_214-1292    8
mis-ibd-PSIc-mix-mv-Il5M*214-1456_214-1456    7
mis-ibd-PSIc-mix-mv-Il5M*215-1663_215-1663    7
mis-ibd-PSIc-mix-mv-Il5M*214-1522_214-1522    6
mis-ibd-PSIc-mix-mv-Il5M*214-1634_214-1634    6
                                             ..
mis-ibd-PSIc-mix-mv-Il5M*216-3034_216-3034    1
mis-ibd-PSIc-mix-mv-Il5M*216-3035_216-3035    1
mis-ibd-PSIc-mix-mv-Il5M*216-3047_216-3047    1
mis-ibd-PSIc-mix-mv-Il5M*208-2512_208-2512    1
mis-ibd-PSIc-mix-mv-Il5M*216-3249_216-3249    1
Name: count, Length: 328, dtype: int64

In [234]:
df_exp_samples[df_exp_samples["exp_id"].str.contains("B")]

,exp_id
0,R690_B_1660_1
1,R501_B_1001_1A
2,R522_B_1418_3A
3,R601_B_734_3A
4,R589_B_1438_1A
...,...
659,R357_B1219_3B
660,R160_B278_1
661,R296_B1481_3A
662,R646_B_1754_2


In [195]:
# create exp_id column
merged_df_final['exp_id'] = 'R' + merged_df_final['Novogene ID'].str.replace('Biopsy', 'B').str.replace('-', '_')

#  Filter out rows where exp_id is R + "n.v.t."
df_filtered = merged_df_final[merged_df_final['exp_id'] != "Rn.v.t."]

# Drop duplicates in the 'geno' column, keeping the first instance
df_unique = df_filtered.drop_duplicates(subset='geno', keep='first')


## extra steps to get more samples, in cases where the exp_id = Bn instead of B_n (e.g. R392_B408_2 instead of R392_B_408_2)
# Find rows where exp_id is not in exp_samples
df_wrong = df_unique[~df_unique["exp_id"].isin(exp_samples)]

# Function to remove the first underscore after "B"
def remove_first_underscore_b(value):
    return re.sub(r'(B)_(.*?)_', r'\1\2_', value)

# Apply the function to exp_id in the filtered DataFrame
df_wrong['exp_id'] = df_wrong['exp_id'].apply(lambda x: remove_first_underscore_b(x) if 'B_' in x else x)

merged_df_corrected = pd.merge(
    df_unique,
    df_wrong,
    on="exp_id",
    how="right"
)

merged_df_corrected.dropna(inplace=True, axis=1)

/tmp/ipykernel_2898376/1515860970.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wrong['exp_id'] = df_wrong['exp_id'].apply(lambda x: remove_first_underscore_b(x) if 'B_' in x else x)


In [196]:
# total amount of unique patients
df_unique["exp_id"].size

216

In [197]:
# total amount of unique patients after merging out the exp_id's that don't overlap 
merged_df_corrected["exp_id"].size

161

In [198]:
# total amount of unique patients that overlap (before tweaking the B_n)
df_unique["exp_id"][df_unique["exp_id"].isin(exp_samples)].size

55

In [199]:
# total amount of unique patients that don't overlap (before tweaking the B_n)
df_unique["exp_id"][~df_unique["exp_id"].isin(exp_samples)].size

161

In [200]:
# total amount of unique patients that overlap (after tweaking B_n -> bn)
merged_df_corrected["exp_id"][merged_df_corrected["exp_id"].isin(exp_samples)].size

151

In [201]:
# total amount of unique patients that don't overlap (after tweaking B_n -> bn)
merged_df_corrected["exp_id"][~merged_df_corrected["exp_id"].isin(exp_samples)].size

10

In [202]:
# samples that don't overlap with the expression data
merged_df_corrected[~merged_df_corrected["exp_id"].isin(exp_samples)]

,exp_id,geno_y,geno_id_y,DNA sample ID_y,Research ID_y,scopy date_y,biopsy research number_y,Novogene ID_y,Cohort_y,Sequencing batch_y
7,R247_B1060_1A,mis-ibd-PSIc-mix-mv-Il5M*210-14215_210-14215,210-14215,210-14215,UMCGIBD00960,13-05-2011,1060-1A,247_Biopsy_1060-1A,1000IBD selection,Novogene 2020
24,R392_B408_2,mis-ibd-PSIc-mix-mv-Il5M*211-0006_211-0006,211-0006,211-0006,UMCGIBD01283,05/12/2005,408-2,392_Biopsy_408-2,1000IBD selection,Novogene 2020
34,R1_B1075_2B,mis-ibd-PSIc-mix-mv-Il5M*214-1291_214-1291,214-1291,214-1291,UMCGIBD00382,29-07-2011,1075-2B,1_Biopsy_1075-2B,1000IBD selection,Novogene 2020
36,R403_B991_2B,mis-ibd-PSIc-mix-mv-Il5M*214-1321_214-1321,214-1321,214-1321,UMCGIBD00474,13-09-2010,991-2B,403_Biopsy_991-2B,1000IBD selection,UMCG 2016 + Novogene 2020
41,R672_B944_3A,mis-ibd-PSIc-mix-mv-Il5M*214-1347_214-1347,214-1347,214-1347,UMCGIBD00117,19-04-2010,944-3A,672_Biopsy_944-3A,1000IBD selection,Novogene 2020
69,R278_B1017_3A,mis-ibd-PSIc-mix-mv-Il5M*214-1547_214-1547,214-1547,214-1547,UMCGIBD00520,13-12-2010,1017-3A,278_Biopsy_1017-3A,1000IBD selection,Novogene 2020
81,R165_B781_1A,mis-ibd-PSIc-mix-mv-Il5M*214-1628_214-1628,214-1628,214-1628,UMCGIBD00333,17-11-2008,781-1A,165_Biopsy_781-1A,1000IBD selection,Novogene 2020
112,R108_B1621_1A,mis-ibd-PSIc-mix-mv-Il5M*215-1620_215-1620,215-1620,215-1620,UMCGIBD01344,17-09-2018,1621-1A,108_Biopsy_1621-1A,1000IBD selection,Novogene 2020
128,R2_B608_2,mis-ibd-PSIc-mix-mv-Il5M*215-1742_215-1742,215-1742,215-1742,UMCGIBD00386,15-02-2007,608-2,2_Biopsy_608-2,1000IBD selection,Novogene 2020
132,R486_B1104_2A,mis-ibd-PSIc-mix-mv-Il5M*215-1768_215-1768,215-1768,215-1768,UMCGIBD00017,19-12-2011,1104-2A,486_Biopsy_1104-2A,1000IBD selection,UMCG 2016 + Novogene 2020


In [230]:
test = "900"
print("merged_df:", df_unique[["geno_id", "exp_id"]][df_unique["exp_id"].str.contains(test)], "\n")
print("old_g2e:",old_g2e[old_g2e["geno"].str.contains(test)], "\n")
print("exp file:",df_exp_samples[df_exp_samples["exp_id"].str.contains(test)], "\n")

merged_df:     geno_id         exp_id
7  209-0075  R172_B_900_2A 

old_g2e:           exp          geno    study
148  209-0075  R172_B900_2A  1000IBD 

exp file:            exp_id
475   R74_B900_1A
642  R172_B900_2A 



In [238]:
merged_df_final

,geno,geno_id,DNA sample ID,Research ID,scopy date,biopsy research number,Novogene ID,Biopt location,Cohort,Sequencing batch,Opmerking,exp_id
0,mis-ibd-PSIc-mix-mv-Il5M*208-2512_208-2512,208-2512,208-2512,UMCGIBD01036,03/03/2008,713-1B,367_Biopsy_713-1B,NO transversum,1000IBD selection,Novogene 2020,NaN,R367_B_713_1B
1,mis-ibd-PSIc-mix-mv-Il5M*209-0014_209-0014,209-0014,209-0014,UMCGIBD00912,08-11-2010,1004-1A,11_Biopsy_1004-1A,NO ileum,1000IBD selection,Novogene 2020,NaN,R11_B_1004_1A
2,mis-ibd-PSIc-mix-mv-Il5M*209-0014_209-0014,209-0014,209-0014,UMCGIBD00912,08-11-2010,1004-2A,154_Biopsy_1004-2A,NO ascendens,1000IBD selection,Novogene 2020,NaN,R154_B_1004_2A
3,mis-ibd-PSIc-mix-mv-Il5M*209-0014_209-0014,209-0014,209-0014,UMCGIBD00912,08-11-2010,1004-3A,81_Biopsy_1004-3A,O sigmoid,1000IBD selection,Novogene 2020,NaN,R81_B_1004_3A
4,mis-ibd-PSIc-mix-mv-Il5M*209-0035_209-0035,209-0035,209-0035,UMCGIBD00599,08/03/2007,659-4A,171_Biopsy_659-4A,O descendens,1000IBD selection,Novogene 2020,NaN,R171_B_659_4A
...,...,...,...,...,...,...,...,...,...,...,...,...
685,mis-ibd-PSIc-mix-mv-Il5M*216-3244_216-3244,216-3244,216-3244,UMCGIBD01532,09-05-2016,1393-3A,52_Biopsy_1393-3A,O rectum/sigmoid,VEDO Valerie Colij,Novogene 2020,NaN,R52_B_1393_3A
686,mis-ibd-PSIc-mix-mv-Il5M*216-3244_216-3244,216-3244,216-3244,UMCGIBD01532,09-05-2016,1393-1A,554_Biopsy_1393-1A,O ileum,VEDO Valerie Colij,Novogene 2020,NaN,R554_B_1393_1A
687,mis-ibd-PSIc-mix-mv-Il5M*216-3247_216-3247,216-3247,216-3247,UMCGIBD01422,29-01-2019,1673-1,351_Biopsy_1673-1,O coecum,Inf/NotInf Werna,Novogene 2020,NaN,R351_B_1673_1
688,mis-ibd-PSIc-mix-mv-Il5M*216-3247_216-3247,216-3247,216-3247,UMCGIBD01422,29-01-2019,1673-2,519_Biopsy_1673-2,NO ascendens,Inf/NotInf Werna,Novogene 2020,NaN,R519_B_1673_2


In [213]:
df_unique

,geno,geno_id,DNA sample ID,Research ID,scopy date,biopsy research number,Novogene ID,Biopt location,Cohort,Sequencing batch,Opmerking,exp_id
0,mis-ibd-PSIc-mix-mv-Il5M*208-2512_208-2512,208-2512,208-2512,UMCGIBD01036,03/03/2008,713-1B,367_Biopsy_713-1B,NO transversum,1000IBD selection,Novogene 2020,NaN,R367_B_713_1B
1,mis-ibd-PSIc-mix-mv-Il5M*209-0014_209-0014,209-0014,209-0014,UMCGIBD00912,08-11-2010,1004-1A,11_Biopsy_1004-1A,NO ileum,1000IBD selection,Novogene 2020,NaN,R11_B_1004_1A
4,mis-ibd-PSIc-mix-mv-Il5M*209-0035_209-0035,209-0035,209-0035,UMCGIBD00599,08/03/2007,659-4A,171_Biopsy_659-4A,O descendens,1000IBD selection,Novogene 2020,NaN,R171_B_659_4A
7,mis-ibd-PSIc-mix-mv-Il5M*209-0075_209-0075,209-0075,209-0075,UMCGIBD00889,12-01-2010,900-2A,172_Biopsy_900-2A,O sigmoid,1000IBD selection,Novogene 2020,NaN,R172_B_900_2A
9,mis-ibd-PSIc-mix-mv-Il5M*209-0132_209-0132,209-0132,209-0132,UMCGIBD00709,19-05-2014,1233-1A,599_Biopsy_1233-1A,NO neo terminale ileum,1000IBD selection,Novogene 2020,NaN,R599_B_1233_1A
...,...,...,...,...,...,...,...,...,...,...,...,...
678,mis-ibd-PSIc-mix-mv-Il5M*216-3235_216-3235,216-3235,216-3235,UMCGIBD01460,26-10-2004,335-1,393_Biopsy_335-1,NO ileum,1000IBD selection,Novogene 2020,NaN,R393_B_335_1
680,mis-ibd-PSIc-mix-mv-Il5M*216-3237_216-3237,216-3237,216-3237,UMCGIBD01432,04-04-2016,1386-1A,547_Biopsy_1386-1A,NO sigmoid,1000IBD selection,Novogene 2020,NaN,R547_B_1386_1A
681,mis-ibd-PSIc-mix-mv-Il5M*216-3243_216-3243,216-3243,216-3243,UMCGIBD01450,19-09-2016,1428-1A,344_Biopsy_1428-1A,NO ileum,1000IBD selection,Novogene 2020,NaN,R344_B_1428_1A
684,mis-ibd-PSIc-mix-mv-Il5M*216-3244_216-3244,216-3244,216-3244,UMCGIBD01532,09-05-2016,1393-2A,22_Biopsy_1393-2A,NO ascendens,VEDO Valerie Colij,Novogene 2020,NaN,R22_B_1393_2A


In [212]:
merged_df_corrected

,exp_id,geno_y,geno_id_y,DNA sample ID_y,Research ID_y,scopy date_y,biopsy research number_y,Novogene ID_y,Cohort_y,Sequencing batch_y
0,R367_B713_1B,mis-ibd-PSIc-mix-mv-Il5M*208-2512_208-2512,208-2512,208-2512,UMCGIBD01036,03/03/2008,713-1B,367_Biopsy_713-1B,1000IBD selection,Novogene 2020
1,R11_B1004_1A,mis-ibd-PSIc-mix-mv-Il5M*209-0014_209-0014,209-0014,209-0014,UMCGIBD00912,08-11-2010,1004-1A,11_Biopsy_1004-1A,1000IBD selection,Novogene 2020
2,R171_B659_4A,mis-ibd-PSIc-mix-mv-Il5M*209-0035_209-0035,209-0035,209-0035,UMCGIBD00599,08/03/2007,659-4A,171_Biopsy_659-4A,1000IBD selection,Novogene 2020
3,R172_B900_2A,mis-ibd-PSIc-mix-mv-Il5M*209-0075_209-0075,209-0075,209-0075,UMCGIBD00889,12-01-2010,900-2A,172_Biopsy_900-2A,1000IBD selection,Novogene 2020
4,R216_B434_1,mis-ibd-PSIc-mix-mv-Il5M*209-0148_209-0148,209-0148,209-0148,UMCGIBD01071,21-09-2005,434-1,216_Biopsy_434-1,1000IBD selection,Novogene 2020
...,...,...,...,...,...,...,...,...,...,...
156,R186_B1416_1A,mis-ibd-PSIc-mix-mv-Il5M*216-3219_216-3219,216-3219,216-3219,UMCGIBD01426,25-07-2016,1416-1A,186_Biopsy_1416-1A,VEDO Valerie Colij,Novogene 2020
157,R393_B335_1,mis-ibd-PSIc-mix-mv-Il5M*216-3235_216-3235,216-3235,216-3235,UMCGIBD01460,26-10-2004,335-1,393_Biopsy_335-1,1000IBD selection,Novogene 2020
158,R344_B1428_1A,mis-ibd-PSIc-mix-mv-Il5M*216-3243_216-3243,216-3243,216-3243,UMCGIBD01450,19-09-2016,1428-1A,344_Biopsy_1428-1A,1000IBD selection,Novogene 2020
159,R22_B1393_2A,mis-ibd-PSIc-mix-mv-Il5M*216-3244_216-3244,216-3244,216-3244,UMCGIBD01532,09-05-2016,1393-2A,22_Biopsy_1393-2A,VEDO Valerie Colij,Novogene 2020


## create link file

In [236]:
new_g2e = merged_df_corrected[["geno_y", "exp_id"]]
new_g2e["study"] = old_g2e["study"]
new_g2e

/tmp/ipykernel_2898376/1227305427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_g2e["study"] = old_g2e["study"]


,geno_y,exp_id,study
0,mis-ibd-PSIc-mix-mv-Il5M*208-2512_208-2512,R367_B713_1B,1000IBD
1,mis-ibd-PSIc-mix-mv-Il5M*209-0014_209-0014,R11_B1004_1A,1000IBD
2,mis-ibd-PSIc-mix-mv-Il5M*209-0035_209-0035,R171_B659_4A,1000IBD
3,mis-ibd-PSIc-mix-mv-Il5M*209-0075_209-0075,R172_B900_2A,1000IBD
4,mis-ibd-PSIc-mix-mv-Il5M*209-0148_209-0148,R216_B434_1,1000IBD
...,...,...,...
156,mis-ibd-PSIc-mix-mv-Il5M*216-3219_216-3219,R186_B1416_1A,1000IBD
157,mis-ibd-PSIc-mix-mv-Il5M*216-3235_216-3235,R393_B335_1,1000IBD
158,mis-ibd-PSIc-mix-mv-Il5M*216-3243_216-3243,R344_B1428_1A,1000IBD
159,mis-ibd-PSIc-mix-mv-Il5M*216-3244_216-3244,R22_B1393_2A,1000IBD


In [219]:
old_g2e

,exp,geno,study
0,214-1378,R103_B1289_1A,1000IBD
1,214-1453,R467_B_815_1B,1000IBD
2,214-1686,R161_B586_1,1000IBD
3,214-1405,R112_B1165_1A,1000IBD
4,214-1800,R575_B_695_3A,1000IBD
...,...,...,...
222,215-1719,R489_B_1345_2A,1000IBD
223,215-1610,R40_B222_1,1000IBD
224,215-1739,R138_B828_2A,1000IBD
225,216-3164,R422_B_431_2A,1000IBD


In [237]:
new_g2e.to_csv("/groups/umcg-fg/tmp04/projects/gut-bulk/ongoing/2024-02-07-GutPublicRNASeq/datasets/Werna/paper_genotype/1000IBD_new_g2e_dedup.tsv", sep="\t", index=False, header=False)